# Cvičenie 6: Neurónové siete v Tensorflow a Keras

Minulý týždeň ste už videli príklad implementácie neurónovej siete pomocou knižnice Keras. Na dnešnom cvičení sa pozrieme na to, ako presne funguje Keras v kombinácii s Tensorflowom. Aby ste úspešne zvládli cvičenie, potrebujete mať nainštalované knižnice Tensorflow (verzia 2) a Keras (verzia 2).

## 1. Výpočty v TensorFlowe

Názov TensorFlow popisuje spôsob vykonávania výpočtov v tomto softvérovom rámci. Tensory sú vlastne údaje (hodnoty alebo viacdimenzionálne polia) a výpočty predstavujú *flow* týchto dát. Na začiatok zadefinujeme jednoduchú operáciu sčítania pomocou TensorFlow:

![](sources/lab06/6.1-addition.png)

In [ ]:
import tensorflow as tf

# create the nodes in the graph, and initialize values
a = tf.constant(13, name="a")
b = tf.constant(37, name="b")

# add together the two values
c = tf.add(a, b, name="c")
print(c)

Výstup výpočtu je tensor s hodnotou 50: `tf.Tensor(50, shape=(), dtype=int32)`. Tensorflow podporuje tensory s hodnotami (môžu byť konštanty alebo premenné) a s výpočtami.

Na základe predošlého príkladu vytvorte viac zložitý graf:

![](sources/lab06/6.2-complicated-graph.png)

In [ ]:
# create the nodes in the graph, and initialize values
a = tf.constant(2.5, name="a")
b = tf.constant(6.5, name="b")

c = # TODO
d = # TODO
e = # TODO

print(e)

## 2. Neurón v TensorFlowe

Neurón predstavuje základný výpočtový prvok neurónových sietí, ktorý pozostáva zo vstupov, synaptických váh, váženej sumy, aktivačnej funkcie a výstupnej funckie. V tomto kroku implementujeme jednoduchý neurón pomocou základných metód TensorFlow, aby ste videli, ako tieto výpočty fungujú na úrovni s nižšou abstrakciou.

![](sources/lab06/6.3-perceptron.png)

In [ ]:
# simple perceptron with two input nodes
def my_neuron(x):
    # define some arbitrary weights for the two input values
    W = tf.constant([[3, -2]], shape=(1, 2), dtype=tf.float32)

    # define the bias of the neuron
    b = 1
    
    # compute weighted sum (hint: check out tf.matmul)
    z = #TODO

    # apply the sigmoid activation function (hint: use tf.sigmoid)
    output = #TODO

    return output

sample_input = tf.constant([[-1, 2]], shape=(1, 2), dtype=tf.float32)

# if you've done everything correctly, this should give you a tensor with value 0.002
result = my_neuron(sample_input)
print(result)

## 3. Plne prepojené vrstvy v TensorFlowe

Ak chceme vytvoriť neurónovú sieť, jednoduché neuróny musíme usporiadať do tzv. vrstiev. Oproti predošlému príkladu musíme tiež upraviť váhy tak, aby boli aktualizovateľné. V neurónových sieťach sa najčastejšie používajú plne prepojené vrstvy (*fully connected* alebo *dense*).

In [ ]:
# x: input values
# n_in: number of input nodes
# n_out: number of output nodes
def my_dense_layer(x, n_in, n_out):
    # define variable weights as a matrix and biases
    # initialize weights for one
    # initialize biases for zero
    W = tf.Variable(tf.ones((n_in, n_out)))
    b = # TODO
    
    # compute weighted sum (hint: check out tf.matmul)
    z = #TODO

    # apply the sigmoid activation function (hint: use tf.sigmoid)
    output = #TODO

    return output

Ako aj pred tým, naše riešenie vieme otestovať zadaním ľubovoľných hodnôt (s dodržaním počtu vstupných a výstupných neurónov).

In [ ]:
sample_input = tf.constant([[1, 2.]], shape=(1, 2))
print(my_dense_layer(sample_input, n_in=2, n_out=3))

Podobne by sme postupovali pri definícii neurónovej siete s viacerými vrstvami, vidíme ale, že takýto zápis je zdĺhavý a neprehľadný. Práve preto bola vyvinutá knižnica Keras, ktorá pridá vrstvu abstrakcie a pomocou nej vieme jednoduchšie pracovať s neurónovými sieťami. Novšie verzie TensorFlowu už obsahujú túto funkcionalitu pod modulom `tf.keras`, ktorý je často rýchlejší ako samotná knižnica `keras`.

## 4. Definícia a trénovanie neurónovej siete pomocou Kerasu

Pre dnešnú ukážku môžete používať ľubuvoľný dataset, odporúčame ale použiť niektorý dataset pre klasifikáciu z [tohto repozitára](https://archive.ics.uci.edu/ml/datasets.php).

Ako sme už videli minulý týždeň, v Kerase potrebujeme vytvoriť model, pridať doňho vrstvy, následne model skompilovať a nakoniec natrénovať. Dnes ukážeme trocha iný prístup, kde najprv zadefinujete vrstvy a prepojíte ich, až potom vytvoríte model. Oba prístupy sú rovnocenné a môžete ich používať ľubovoľne. Najčastejšie sa používa pridávanie vrstiev priamo do modelu, layers-first prístup môžete používať vtedy, ak sieť chcete trénovať naraz, ale jej časti chcete používať nezávisle (napr. autoenkódery, GAN siete, atď.), alebo potrebujete zadefinovať viac vetiev v neurónovej sieti.

In [ ]:
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

input_layer = Input(shape=(4,))
hidden_layer = Dense(10, activation='tanh')(input_layer)
output_layer = Dense(3, activation='softmax')(hidden_layer)

model = Model(input_layer, output_layer)
model.summary()

Pri tomto prístupe nezadefinujete presný typ siete, resp. modelu, používate všeobecnú implementáciu `Model`, ktorý definujú vstupné a výstupné vrstvy; architektúra siete je daná prepojením vrstiev.

Prvá vrstva v sieti musí byť typu `Input`, ktorý reprezentuje vstupnú vrstvu. Vstupná vrstva nemá žiadnu aktivačnú funkciu, svoj vstup iba posunie do ďalšej vrstvy bez zmien. Konštruktor vrstvy má jeden povinný parameter, čo je tvar vstupu (zároveň počet neurónov). V našom príklade definujeme sieť so štyrmi vstupnými neurónmi. Podrobnejší popis `Input` vrstvy so všetkými parametrami nájdete v [dokumentácii Kerasu](https://keras.io/api/layers/core_layers/input/).

Ďalšie dve vrstvy sú typu `Dense`, sú teda plne prepojené vrstvy (každý neurón je prepojený s každým neurónom z predošlej vrstvy). Konštruktor inicializuje váhy a biasy, definuje aktivačnú funkciu ako aj počet neurónov. Takisto môžete definovať tvar vstupu, najlepšie ale je keď to necháte na knižnicu tak, že vytvorenú vrstvu prepojíte s predošlou vrstvou. To urobíte odovzdaním predošlej vrstvy ako parameter (v zátvorkách na konci riadku). V konštruktore viete určiť, či chcete používať biasy, ako ich má modul inicializovať a takisto spôsob inicializácie váh. Ďalšie informácie nájdete v [dokumentácii Kerasu](https://keras.io/api/layers/core_layers/dense/).

Čo sa týka výberu aktivačných funkcií, odporúčame nasledovné heuristické pravidlá (to znamená, že zvyčajne sú efektívne, ale nie vždy):
* pre regresiu používajte aktivačnú funkciu `linear`
* pre klasifikáciu používajte
  * `ReLU` alebo `tanh` v skrytých vrstvách
  * vo výstupnej vrstve `sigmoid` pre binárnu klasifikáciu a `softmax` pre klasifikáciu do viacerých tried

Užitočná metóda je metóda `summary`, ktorá vám vypíše informácie o definovanej sieti. Napríklad pre vyšší príklad metóda vygeneruje nasledovný výstup:

In [ ]:
model.summary()

```
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
=================================================================
Total params: 83
Trainable params: 83
Non-trainable params: 0
```

Z tabuľky viete zistiť názov a typ jednotlivých vrstiev, tvar ich výstupu, počet parametrov ako aj celkový počet parametrov pre celú sieť. Keras takisto umožňuje vypnutie možnosti trénovania pre niektoré parametre (`non-trainable params`).

Ak ste spokojní s topológiou siete, model môžete skompilovať už známym spôsobom - pre zoznam a vysvetlenie parametrov metódy vám odporúčame prečítať [dokumentáciu](https://keras.io/api/models/model_training_apis/#compile-method):

In [ ]:
model.compile(Adam(lr=0.04), 'categorical_crossentropy', metrics=['accuracy'])

Trénovanie sa uskutoční zavolaním metódy `fit` - podrobný popis nájdete v [dokumentácii](https://keras.io/api/models/model_training_apis/#fit-method).

Z hľadiska rýchlosti trénovania a konvergencie je kľúčová hodnota `batch_size`: batch je malá skupina príkladov z trénovacej množiny na základe ktorých sa aktualizujú váhy a biasy. Keďže aktualizácia je výpočtovo náročná operácia, jeho vykonávanie po každom príklade (`batch_size = 1`) by bolo pomalé, aj keď konvergencia by bola optimálna. Na druhej strane, ak aktualizáciu vykonáte až po epoche (`batch_size = len(train_X)`, prechádzanie celou trénovacou množinou), vykonávanie síce bude rýchle, ale konvergencia bude neoptimálna. Najlepšie je vybrať si dostatočne nízku hodnotu, aby vykonávanie bolo rýchlejšie, ale aby ste neprišli o cenné informácie pre konvergenciu. Pomocou veľkosti batchu takisto viete šetriť pamäť počítača, resp. grafickej karty.

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Ak chcete natrénovanú sieť používať, potrebujete zavolať metódu `predict` ([dokumentácia](https://keras.io/api/models/model_training_apis/#predict-method)):

In [ ]:
y_pred = model.predict(X_test)

**Poznámka**: ukážkové riešenie úloh s TensorFlowom nájdete [tu](sources/lab06/lab6-tensorflow-solution.py)).